# SHAP with XGBoost

In this notebook, we will use the Titanic dataset to predict the survival of passengers using XGBoost. We will use SHAP to explain the predictions of the models.

### Install and import the necessary libraries

If you're running this on Google Colab, you'll need to install Dalex and SHAP but the other libraries should already be installed. If you're running this on your local machine, you'll need to install all the libraries needed.

In [ ]:
# Install the necessary libraries

# !pip install -q dalex xgboost shap

In [ ]:
# Import the necessary libraries
import dalex as dx
import xgboost as xgb
import shap

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

### Load the Titanic dataset

The Titanic dataset is a popular toy dataset for machine learning. It contains records about passengers on the Titanic, including whether they survived (i.e. `survived` = 1) or not (i.e. `survived` = 0). While the larger dataset contains more information, we are using a smaller version that only contains these specific columns:

- `gender`: passengers' recorded gender.
- `age`: passengers' age in years.
- `class`: passengers' ticket class on the Titanic, which can be 1st, 2nd, or 3rd class, or a number of crew-specific classes.
- `embarked`: the port where the passenger embarked from, which can be Cherbourg, Queenstown, or Southampton.
- `fare`: how much the passenger paid for their ticket in British pounds.
- `sibsp`: the number of siblings or spouses the passenger had on board.
- `parch`: the number of parents or children the passenger had on board.
- `survived`: whether the passenger survived or not.

In [ ]:
df = dx.datasets.load_titanic()
df.head()

We can get some programmatic information about the dataset using the `info` method. In our uses, we will need to be careful about how we handle the categorical columns, which are the columns with the 'object' dtype. In our case, this is the `gender`, `class`, and `embarked` columns. For `gender`, we will shortly be replacing this with a 0-1 encoding. For `class` and `embarked`, we will convert the columns to a 'category' dtype so that they can be used in the model. Later we will need to use one-hot encoding for these features, where each possible value gets its own column.

In [ ]:
df.info()

The columns labelled 'object' as their dtype are categorical columns. We can convert them into a 'category' dtype so that they can be used in the model.

In [ ]:
df['class'] = df['class'].astype('category')
df['embarked'] = df['embarked'].astype('category')

In [ ]:
df.info()

### Split the data into features and target

We will also use the 'get_dummies' method to convert gender to a 0-1 encoding.

In [ ]:
X = df.drop('survived', axis=1)
y = df['survived']

X = pd.get_dummies(X, columns=['gender'], drop_first=True)

In [ ]:
X.head()

### Split the data into training and testing sets

As always in machine learning, we need to hold back some of our data for testing. We will use 80% of the data for training and 20% for testing. We fix the random state so that the results are reproducible - this means that our randomness is not truly random, but it is consistent across runs.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Create our XGBoost model

While we won't be covering XGBoost in this tutorial, it's a popular machine learning library that implements gradient boosting. In a nutshell, this is a type of ensemble learning where a series of decision trees are trained sequentially, with each tree trying to correct the errors of the previous tree. This is a powerful technique that can be used for both regression and classification tasks.

In [ ]:
model = xgb.XGBClassifier(
    n_estimators = 200,        # Number of trees to fit
    max_depth = 4,             # Maximum tree depth for individual trees
    use_label_encoder = False, # Leave this as False to avoid warnings
    enable_categorical = True,  # Leave this as True to use categorical columns
    tree_method = 'hist'       # Use a histogram-based method for faster training
)

In [ ]:
model.fit(X_train, y_train)

We can quickly check the accuracy of our model using the `score` method. This method calculates the accuracy of the model on the test set. You should expect to see a bit under 80% accuracy.

In [ ]:
model.score(X_test, y_test)

### Explain with Dalex

[Dalex](https://dalex.drwhy.ai/) is a library that helps you understand how machine learning models work. It provides tools for understanding the model's performance, the importance of features, and how the model makes predictions. We can use Dalex to explain the predictions of our XGBoost model.

We are going to create an instance of the `Explainer` class, which gives us access to a range of methods for understanding the model. To create this, we need to specify how the explainer can get predictions from the model. It expects to receive a single value for each entry that corresponds to the likelihood of the positive class, so that's exactly what our code does. We also specify the label of the model as 'XGBoost'.

In [ ]:
def pf_xgboost_classifier_categorical(model, df):
    # Make absolutely sure that the categorical columns are of type 'category'
    df.loc[:, df.dtypes == 'object'] = \
        df.select_dtypes(['object']) \
            .apply(lambda x: x.astype('category'))
    # Predict the probability of the positive class
    return model.predict_proba(df)[:, 1]

explainer = dx.Explainer(model, X, y, predict_function=pf_xgboost_classifier_categorical, label='XGBoost')

We can do a range of useful things with our explainer now, including getting an overview of our model's performance:

In [ ]:
explainer.model_performance()

...making predictions:

In [ ]:
explainer.predict(X_test[0:10])

...and most important for this lab, explaining the predictions of the model. Here we will get the explanations using SHAP for the first 5 passengers in the data.

In [ ]:
shap_attributions = [explainer.predict_parts(X.iloc[[i]], type="shap", label=f'passenger {i}') for i in range(5)]

In [ ]:
shap_attributions[0].plot(shap_attributions[1::])

The above plots show us each feature's contribution to the final prediction for that instance. Below we will see a different visualization of the same data, where this time we track how the prediction changes as we add more features.

In [ ]:
bd_attributions = [explainer.predict_parts(X.iloc[[i]], type="break_down", label=f'passenger {i}') for i in range(5)]

In [ ]:
bd_attributions[0].plot(bd_attributions[1::])

### Explain with SHAP

In addition to the Dalex library we used above, we can also use the original SHAP library to make the same type of interpretations. Here we will need to use one-hot encoding instead of our categorical dtype. The code below handles this for us.

Note that `drop_first=True` means that we don't have a column for one of the possible values - instead, this value is inferred by having `False` in all the other columns for that feature. This improves our data efficiency slightly, but is a problem if we have missing data (since that would also be represented by a series of `False` values). Luckily, we have no missing data in this version of the Titanic dataset, although the larger version does have missing data.

In [ ]:
# Need to convert the categorical columns to one-hot encoding
X_ohe = pd.get_dummies(X, columns=['class', 'embarked'], drop_first=True)

In [ ]:
X_ohe.head()

Now let's build another classifier using the one-hot encoded data, for us to examine the SHAP values.

In [ ]:
X_train_ohe, X_test_ohe, y_train, y_test = train_test_split(X_ohe, y, test_size=0.2, random_state=42)

model_ohe = xgb.XGBClassifier(
    n_estimators = 200,        # Number of trees to fit
    max_depth = 4,             # Maximum tree depth for individual trees
    use_label_encoder = False, # Leave this as False to avoid warnings
    tree_method = 'hist'       # Use a histogram-based method for faster training
)

In [ ]:
model_ohe.fit(X_train_ohe, y_train)

In [ ]:
model_ohe.score(X_test_ohe, y_test)

Interestingly, this model performs ever so slightly better than the previous model. Could this be due to a difference in the random split? Or is it because the one-hot encoding is more effective for this dataset and model combination? We can't say for sure, but it's worth considering.

In [ ]:
# The SHAP explainer really doesn't like converting datatypes for us, so we are converting everything to a float here
explainer_ohe = shap.explainers.Tree(model_ohe, data=X_train_ohe.astype('float64'), model_output='probability')

In [ ]:
shap_values = explainer_ohe.shap_values(X_test_ohe)

In [ ]:
shap.summary_plot(shap_values, X_test_ohe)

In [ ]:
for i in range(5):
    shap.force_plot(explainer_ohe.expected_value, shap_values[i], X_test_ohe.iloc[i], matplotlib=True)